In [93]:
# good example - https://python.langchain.com/docs/use_cases/more/agents/agents/sales_agent_with_context
# - learn what a LLMCHain is
# - PromptTemplate
# - OpenAIEmbeddings
# - Chroma = embeddings, texts 
# - Retrieval QA
# - Tool
# - class SalesGPT(Chain, BaseModel):

In [ ]:
# - normandy

: 

: 

good example - https://python.langchain.com/docs/use_cases/more/agents/agents/sales_agent_with_context
- learn what a LLMCHain is
- PromptTemplate
- OpenAIEmbeddings
- Chroma = embeddings, texts 
- Retrieval QA
- Tool
- class SalesGPT(Chain, BaseModel):

agent example - https://python.langchain.com/docs/modules/agents/

- normandy docs: https://rcm-api.athelas.com/api/docs
- authenticate w/ /v1/auth/token endpoint 

LLMChain - https://python.langchain.com/docs/modules/chains/foundational/llm_chain

In [ ]:
# good articles about ReAct LLM architecture
# - https://www.geoffreylitt.com/2023/01/29/fun-with-compositional-llms-querying-basketball-stats-with-gpt-3-statmuse-langchain.html
# - https://interconnected.org/home/2023/03/16/singularity
# - https://github.com/ysymyth/ReAct/blob/master/WebShop.ipynb
# - https://til.simonwillison.net/llms/python-react-pattern

# helpful llm stuff
# - https://thakkarparth007.github.io/copilot-explorer/posts/copilot-internals.html
# - https://gist.github.com/rain-1/eebd5e5eb2784feecf450324e3341c8d
# - https://news.ycombinator.com/item?id=36409489
# - https://github.com/nat/openplayground
# - https://magazine.sebastianraschka.com/p/why-the-original-transformer-figure
# - https://github.com/mpaepper/llm_agents

: 

: 

# Setting up virtual env
<!-- - https://janakiev.com/blog/jupyter-virtual-envs/#add-virtual-environment-to-jupyter-notebook -->
<!-- - after creating virtual env, use it as kernel  -->

In [1]:
!pip3 install langchain
!pip3 install wikipedia
!pip install openai
!pip install pandas
!pip install python-dotenv


In [11]:
import os
from dotenv import load_dotenv

load_dotenv(".env")

open_ai_api_key = os.getenv("OPEN_AI_API_KEY")
normandy_db_username = os.getenv("NORMANDY_DB_USERNAME")
normandy_db_local_port = os.getenv("NORMANDY_DB_LOCAL_PORT")

# Basic ReAct example

In [ ]:
from langchain import OpenAI, Wikipedia
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.agents.react.base import DocstoreExplorer

docstore = DocstoreExplorer(Wikipedia())
tools = [
    Tool(
        name="Search",
        func=docstore.search,
        description="useful for when you need to ask with search",
    ),
    Tool(
        name="Lookup",
        func=docstore.lookup,
        description="useful for when you need to ask with lookup",
    ),
]

llm = OpenAI(temperature=0, model_name="text-davinci-002")
react = initialize_agent(tools, llm, agent=AgentType.REACT_DOCSTORE, verbose=True)

: 

: 

In [ ]:
question = "Author David Chanoff has collaborated with a U.S. Navy admiral who served as the ambassador to the United Kingdom under which President?"
react.run(question)



> Entering new AgentExecutor chain...

Thought: I need to search David Chanoff and find the U.S. Navy admiral he collaborated with. Then I need to find which President the U.S. Navy admiral served as the ambassador to the United Kingdom under.

Action: Search[David Chanoff]

Observation: David Chanoff is a noted author of non-fiction work. His work has typically involved collaborations with the principal protagonist of the work concerned. His collaborators have included; Augustus A. White, Joycelyn Elders, Đoàn Văn Toại, William J. Crowe, Ariel Sharon, Kenneth Good and Felix Zandman. He has also written about a wide range of subjects including literary history, education and foreign for The Washington Post, The New Republic and The New York Times Magazine. He has published more than twelve books.
Thought: The U.S. Navy admiral David Chanoff collaborated with is William J. Crowe. I need to find which President he served as the ambassador to the United Kingdom under.

Action: Search[Wi

'Bill Clinton'

: 

: 

# Basic DB Query example

In [95]:
# conda install postgresql
# conda install psycopg2
import psycopg2

def connect_to_db():
    try:
        connection = psycopg2.connect(
            user="aditya.gunda@getathelas.com",
            password="",
            host="localhost",
            port="54329",
            database="normandy"
        )
        return connection
    except Exception as error:
        print(f"Error while connecting to PostgreSQL: {error}")

def fetch_data(query):
    connection = connect_to_db()
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        records = cursor.fetchall()
        return records
    except Exception as error:
        print(f"Error fetching data: {error}")
    finally:
        cursor.close()
        connection.close()


In [96]:
claim_submissions = fetch_data("SELECT submission_request FROM claim_submissions LIMIT 1000")


In [ ]:
import pandas as pd
df = pd.DataFrame(claim_submissions, columns=['Submission Request'])
df.head()

,Submission Request
0,"{'billing': {'npi': '1306488762', 'address': {..."
1,"{'billing': {'npi': '1568035772', 'address': {..."
2,"{'billing': {'npi': '1386896652', 'address': {..."
3,"{'billing': {'npi': '1568035772', 'address': {..."
4,"{'billing': {'npi': '1568035772', 'address': {..."


: 

: 

In [ ]:
from openai.embeddings_utils import get_embedding

# embedding model parameters
embedding_model = "text-embedding-ada-002"
embedding_encoding = "cl100k_base"  # this the encoding for text-embedding-ada-002
max_tokens = 8000  # the maximum for text-embedding-ada-002 is 8191

df["embedding"] = df['Submission Request'].astype(str).apply(lambda x: get_embedding(x, engine=embedding_model))


: 

: 

In [ ]:
df.to_csv('claim_submissions_embeddings.csv', index=False)

# df = pd.read_csv('claim_submissions_embeddings.csv')

: 

: 

In [ ]:
from openai.embeddings_utils import get_embedding, cosine_similarity

# search through the reviews for a specific product
def search_submissions(df, submission_request, n=50, pprint=True):
    submission_request_embedding = get_embedding(
        submission_request,
        engine="text-embedding-ada-002"
    )
    df["similarity"] = df.embedding.apply(lambda x: cosine_similarity(x, submission_request_embedding))

    results = (
        df.sort_values("similarity", ascending=False)
        .head(n)
    )
    if pprint:
        for r in results:
            print(r)
    
    return results

matched_submissions = search_submissions(
    df, 
    str({ "billing": { "npi": "1356530182", "address": { "city": "NEWARK", "state": "NY", "address1": "513 W UNION ST", "postalCode": "145131365" }, "employerId": "161506371", "providerType": "BillingProvider", "taxonomyCode": "261QP2000X", "organizationName": "BrownStone Physical Therapy PC" }, "receiver": { "organizationName": "NEW YORK BLUE SHIELD OF ROCHESTER" }, "referring": { "npi": "1194762716", "address": { "city": "GENEVA", "state": "NY", "address1": "875 PRE EMPTION RD", "postalCode": "144562042" }, "lastName": "CYWINSKI", "firstName": "DAVID", "providerType": "ReferringProvider", "taxonomyCode": "207RS0010X" }, "rendering": { "npi": "1104132455", "address": { "city": "NEWARK", "state": "NY", "address1": "513 W UNION ST", "postalCode": "145131365" }, "lastName": "Kearns", "firstName": "Matthew", "employerId": "161506371", "providerType": "RenderingProvider", "taxonomyCode": "261QP2000X" }, "submitter": { "organizationName": "BrownStone Physical Therapy PC", "contactInformation": { "name": "Matthew Kearns", "phoneNumber": "+13153313784" } }, "subscriber": { "gender": "M", "address": { "city": "Lyons", "state": "NY", "address1": "10 Maple St", "postalCode": "14489" }, "lastName": "SWEET", "memberId": "VYT204074389", "firstName": "ERIC", "dateOfBirth": "19930917", "paymentResponsibilityLevelCode": "P" }, "controlNumber": "000000001", "claimInformation": { "serviceLines": [ { "serviceDate": "20230220", "renderingProvider": { "npi": "1104132455", "address": { "city": "NEWARK", "state": "NY", "address1": "513 W UNION ST", "postalCode": "145131365" }, "lastName": "Kearns", "firstName": "Matthew", "employerId": "161506371", "providerType": "RenderingProvider", "taxonomyCode": "261QP2000X" }, "professionalService": { "procedureCode": "97140", "measurementUnit": "UN", "serviceUnitCount": "1", "procedureModifiers": [ "GP", "59" ], "procedureIdentifier": "HC", "lineItemChargeAmount": "72.86", "compositeDiagnosisCodePointers": { "diagnosisCodePointers": [ "1" ] } } }, { "serviceDate": "20230220", "renderingProvider": { "npi": "1104132455", "address": { "city": "NEWARK", "state": "NY", "address1": "513 W UNION ST", "postalCode": "145131365" }, "lastName": "Kearns", "firstName": "Matthew", "employerId": "161506371", "providerType": "RenderingProvider", "taxonomyCode": "261QP2000X" }, "professionalService": { "procedureCode": "97110", "measurementUnit": "UN", "serviceUnitCount": "1", "procedureModifiers": [ "GP" ], "procedureIdentifier": "HC", "lineItemChargeAmount": "79.74", "compositeDiagnosisCodePointers": { "diagnosisCodePointers": [ "1" ] } } }, { "serviceDate": "20230220", "renderingProvider": { "npi": "1104132455", "address": { "city": "NEWARK", "state": "NY", "address1": "513 W UNION ST", "postalCode": "145131365" }, "lastName": "Kearns", "firstName": "Matthew", "employerId": "161506371", "providerType": "RenderingProvider", "taxonomyCode": "261QP2000X" }, "professionalService": { "procedureCode": "97530", "measurementUnit": "UN", "serviceUnitCount": "1", "procedureModifiers": [ "GP" ], "procedureIdentifier": "HC", "lineItemChargeAmount": "104.01", "compositeDiagnosisCodePointers": { "diagnosisCodePointers": [ "1" ] } } } ], "claimFilingCode": "BL", "claimChargeAmount": "256.61", "claimFrequencyCode": "1", "placeOfServiceCode": "11", "signatureIndicator": "Y", "patientControlNumber": "ATHELAS-RCM-116300", "planParticipationCode": "A", "releaseInformationCode": "Y", "healthCareCodeInformation": [ { "diagnosisCode": "M5450", "diagnosisTypeCode": "ABK" } ], "benefitsAssignmentCertificationIndicator": "Y" }, "tradingPartnerName": "NEW YORK BLUE SHIELD OF ROCHESTER", "serviceFacilityLocation": { "npi": "1104132455", "address": { "city": "NEWARK", "state": "NY", "address1": "513 W UNION ST STE 254", "postalCode": "145131370" }, "organizationName": "BrownStone Physical Therapy PC" }, "tradingPartnerServiceId": 4408 }),
    n=200
  )

df_2 = pd.DataFrame(matched_submissions, columns=['Submission Request'])
df_2.head(50)
df_2.to_csv('similar_claims.csv', index=False)


Submission Request
embedding
similarity


: 

: 

# Inconsistent Procedure Code / Modifer Issue

In [14]:
import os
from dotenv import load_dotenv

# Make sure to set up a .env file with the following variables
load_dotenv(".env")

open_ai_api_key = os.getenv("OPEN_AI_API_KEY")
normandy_db_username = os.getenv("NORMANDY_DB_USERNAME")
normandy_db_local_port = os.getenv("NORMANDY_DB_LOCAL_PORT")

In [15]:
# conda install postgresql
# conda install psycopg2
import psycopg2

def connect_to_db():
    try:
        connection = psycopg2.connect(
            user="aditya.gunda@getathelas.com",
            password="",
            host="localhost",
            port="54329",
            database="normandy"
        )
        return connection
    except Exception as error:
        print(f"Error while connecting to PostgreSQL: {error}")

def fetch_data(query):
    connection = connect_to_db()
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        records = cursor.fetchall()
        return records
    except Exception as error:
        print(f"Error fetching data: {error}")
    finally:
        cursor.close()
        connection.close()


In [16]:
import json

def lookup_claim_denial_info(claim_id: int):
  def map_data_to_dicts(data):
    keys = [
        "claim_submission_id", "cpt_code", "modifiers", "place_of_service_code",
        "revenue_code", "insurance_payout", "smartpay_pr", 
        "claim_adjustment_reason_codes", "remittance_advice_remark_codes"
    ]
    return [dict(zip(keys, row)) for row in data]

  data = fetch_data("""
          WITH payout AS (
            SELECT
                csp.claim_submission_id,
                csp.procedure_id,
                csp.modifiers,
                v.remit_insurance_priority::text,
                SUM(COALESCE(v.provider_paid_amount_in_usd_cents, 0))::float / 100 as insurance_payout,
                SUM(COALESCE(v.total_adjustments_smartpay_pr_in_usd_cents, 0))::float / 100 as smartpay_pr
            FROM
                claim_submissions_procedures csp
            JOIN
                all_procedure_details_view v ON (
                            v.claim_submission_id = csp.claim_submission_id
                        AND v.procedure_id = csp.procedure_id
                    )
            WHERE
                csp.claim_submission_id = '{claim_id}'
          GROUP BY 1,2,3,4
        )
        , carc_rarc AS (
          SELECT
              pr.procedure_id,
              ARRAY_AGG(CONCAT(carc.code,' - ', carc.description)) as claim_adjustment_reason_codes,
              ARRAY_AGG(CONCAT(rarc.code, ' - ', rarc.description)) as remittance_advice_remark_codes
          FROM
                  claim_submissions cs
          LEFT JOIN
                  claim_reconciliations crecon ON cs.id = crecon.claim_submission_id
          LEFT JOIN
                  claim_remittances rem ON rem.claim_reconciliation_id = crecon.id
          LEFT JOIN
                  procedure_remittances pr ON pr.claim_remittance_id = rem.id
          LEFT JOIN
                  remittance_advice_remark_codes_procedure_remittances rarcpr ON rarcpr.procedure_remittances_id = pr.id
          LEFT JOIN
                  remittance_advice_remark_codes rarc ON rarc.code = rarcpr.remittance_advice_remark_code
          LEFT JOIN
                  procedure_adjustments pa ON pa.procedure_remittance_id = pr.id
          LEFT JOIN
                  claim_adjustment_reason_codes carc ON carc.code = pa.claim_adjustment_reason_code
          WHERE
              cs.id = '{claim_id}'
              AND carc.code NOT IN ('45')
          GROUP BY
              1
        )

        SELECT
            payout.claim_submission_id,
            cc.code as cpt_code,
            payout.modifiers,
            proc.place_of_service_code,
            rc.code as revenue_code,
            payout.insurance_payout,
            payout.smartpay_pr,
            cr.claim_adjustment_reason_codes,
            cr.remittance_advice_remark_codes
        FROM
            payout
        JOIN
            procedures proc ON proc.id = payout.procedure_id
        JOIN
            cpt_codes cc ON cc.id = proc.cpt_code_id
        LEFT JOIN
            revenue_codes rc ON proc.revenue_code_id = rc.id
        LEFT JOIN
            carc_rarc cr ON cr.procedure_id = proc.id;
    """.format(claim_id=claim_id)
  )

  mapped_data = map_data_to_dicts(data)
  return json.dumps(mapped_data, indent=4)


In [17]:
import json 

def lookup_approved_claims_with_cpt_code(cpt_code: str):
  def map_data_to_dicts(data):
    keys = [
        "claim_submission_id", "cpt_codes", "place_of_service_code",
        "revenue_code", "insurance_payout", "smartpay_pr"
    ]
    
    return [dict(zip(keys, row)) for row in data]

  data = fetch_data("""
    WITH relevant_claims AS (
      SELECT
          ARRAY_AGG(DISTINCT claim_submission_id) as claim_submission_ids
      FROM
          claim_submissions_procedures csp
      JOIN
        procedure_payout_view ppv ON ppv.procedure_id = csp.procedure_id
      JOIN
        cpt_codes cc ON csp.cpt_code_id = cc.id
      WHERE
          cc.code = '{cpt_code}'
          AND COALESCE(ppv.provider_paid_amount_in_usd_cents, 0) > 0
    ),
    payout AS (
      SELECT
          csp.claim_submission_id,
          csp.procedure_id,
          csp.modifiers,
          v.remit_insurance_priority::text,
          SUM(COALESCE(v.provider_paid_amount_in_usd_cents, 0))::float / 100 as insurance_payout,
          SUM(COALESCE(v.total_adjustments_smartpay_pr_in_usd_cents, 0))::float / 100 as smartpay_pr
      FROM
          claim_submissions_procedures csp
      JOIN
          all_procedure_details_view v ON (
                  v.claim_submission_id = csp.claim_submission_id
                  AND v.procedure_id = csp.procedure_id
              )
      WHERE
        csp.claim_submission_id IN (SELECT unnest(claim_submission_ids) FROM relevant_claims)
      GROUP BY 1,2,3,4
      HAVING SUM(COALESCE(v.provider_paid_amount_in_usd_cents, 0))::float > 0
      ORDER BY 1 DESC
    ) 

    SELECT
        payout.claim_submission_id,
        ARRAY_AGG(CONCAT(cc.code, '-', payout.modifiers)) as cpt_codes,
        ARRAY_AGG(DISTINCT proc.place_of_service_code) as place_of_service_code,
        ARRAY_AGG(DISTINCT rc.code) as revenue_code,
        SUM(COALESCE(payout.insurance_payout, 0)) as insurance_payout,
        SUM(COALESCE(payout.smartpay_pr, 0)) as smartpay_pr
    FROM
        payout
    JOIN
        procedures proc ON proc.id = payout.procedure_id
    JOIN
        cpt_codes cc ON cc.id = proc.cpt_code_id
    LEFT JOIN
        revenue_codes rc ON proc.revenue_code_id = rc.id
    GROUP BY
      1
    LIMIT 10;
  """.format(cpt_code=cpt_code)
  )

  mapped_data = map_data_to_dicts(data)
  return json.dumps(mapped_data, indent=4)

In [18]:
from langchain import OpenAI
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType

tools = [
    Tool(
        name="Lookup Claim Procedure Denial Info",
        func=lookup_claim_denial_info,
        description="""
          Useful for when you need get a claim's cpt codes and their denial reasons.
          Claims may have multiple procedures.
          The denial reasons are provided in the form of claim adjustment reason codes and remittance advice remark codes.

           Args: 
            claim_id (int): the id of the claim you want to lookup
          Returns:
            list[tuple]: A list containing tuples with procedure data. Each tuple  follows this format: (claim_submission_id, cpt_code, modifiers, place_of_service_code, revenue_code, insurance_payout, smartpay_pr, claim_adjustment_reason_codes, remittance_advice_remark_codes)
        """,
    ),
    Tool(
        name="Lookup Approved Claims where CPT Code was approved",
        func=lookup_approved_claims_with_cpt_code,
        description="""
          Useful to find claims where a certain cpt_code was approved. 
          You can use the claim_submission_id to determine which procedures that are in the same claim.
          
          Args: 
            cpt_code (str): the cpt code you want to search for (e.g. 99214)
          Returns:
            list[tuple]: A list containing tuples with procedure data. Each tuple follows this format: (claim_submission_id, list of cpt_codes followed by their modifiers (e.g. 99214-{{95}}), place_of_service_code, revenue_code, insurance_payout, smartpay_pr)
         """,
    ),
]

PREFIX = """ 
  You are an AI assistant that is trying to solve medical billing claims that were denied by insurers. 
  You will be given a claim submission that was denied. 
  First, use the provided tools to understand why the claim was denied. There may be multiple procedures in a claim, so analyze the procedures with 0 insurance_payout and understand their denial reasons.
  Second, based on the denial reason, look at patterns in similar claims that were approved by insurers (e.g. the modifiers that were used, etc.).
  Finally, use all this information propose a specific solution for the denied claim
"""
llm = OpenAI(temperature=0, model_name="gpt-3.5-turbo-16k")
denials_copilot = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, agent_kwargs={ 'prefix': PREFIX }, verbose=True)

/Users/adityagunda/miniconda3/envs/copilot-playground/lib/python3.11/site-packages/langchain/llms/openai.py:200: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/Users/adityagunda/miniconda3/envs/copilot-playground/lib/python3.11/site-packages/langchain/llms/openai.py:787: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


In [19]:
question = "How can I fix claim 109943?"
denials_copilot.run(question)



> Entering new AgentExecutor chain...
I should start by looking up the denial information for claim 109943 to understand why it was denied.
Action: Lookup Claim Procedure Denial Info
Action Input: 109943
Observation: [
    {
        "claim_submission_id": 109943,
        "cpt_code": "96372",
        "modifiers": [
            "25"
        ],
        "place_of_service_code": "11",
        "revenue_code": null,
        "insurance_payout": 0.0,
        "smartpay_pr": 0.0,
        "claim_adjustment_reason_codes": [
            "4 - The procedure code is inconsistent with the modifier used. Usage: Refer to the 835 Healthcare Policy Identification Segment (loop 2110 Service Payment Information REF), if present."
        ],
        "remittance_advice_remark_codes": [
            " - "
        ]
    }
]
Thought:The claim with ID 109943 was denied because the procedure code 96372 is inconsistent with the modifier used. There are no remittance advice remark codes provided.

Thought: Now I need

'To fix claim 109943, use the modifier "{}" for the procedure code 96372.'

In [20]:
question = "How can I fix claim 275925?"
denials_copilot.run(question)



> Entering new AgentExecutor chain...
I should start by looking up the denial information for claim 275925 to understand why it was denied.
Action: Lookup Claim Procedure Denial Info
Action Input: 275925
Observation: [
    {
        "claim_submission_id": 275925,
        "cpt_code": "99214",
        "modifiers": [
            "25",
            "95"
        ],
        "place_of_service_code": "11",
        "revenue_code": null,
        "insurance_payout": 86.34,
        "smartpay_pr": 0.0,
        "claim_adjustment_reason_codes": null,
        "remittance_advice_remark_codes": null
    },
    {
        "claim_submission_id": 275925,
        "cpt_code": "36415",
        "modifiers": [
            "95"
        ],
        "place_of_service_code": "11",
        "revenue_code": null,
        "insurance_payout": 0.0,
        "smartpay_pr": 0.0,
        "claim_adjustment_reason_codes": [
            "4 - The procedure code is inconsistent with the modifier used. Usage: Refer to the 835 Healt

'To fix claim 275925, I recommend reviewing the inconsistent modifier usage for the procedure with CPT code 36415 and modifier 95. Ensure that the modifier is used correctly according to the guidelines provided by the insurer. Additionally, compare the place of service code and revenue code used in claim 275925 with the approved claims to ensure consistency.'